In [ ]:
from PIL import Image
import numpy as np
import os
import cv2
import glob,random
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical, plot_model
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Dense, Dropout, Activation, Flatten ,BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import keras

In [ ]:
data_dir  = "F:/WorkSpace/Sign_Language_Testing/Words_testing/st_end/data/edge"

In [ ]:
classes   =  os.listdir(data_dir)
print(classes)
print(len(classes))

In [ ]:
count=0
data=[]
labels=[]

In [ ]:
image_files = [f for f in glob.glob(data_dir + "/**/*", recursive=True) if not os.path.isdir(f)]
random.shuffle(image_files)

In [ ]:
for img in image_files:
    count+=1
    image = cv2.resize(cv2.imread(img),(240,240))
    image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image = img_to_array(image)
    data.append(image)
    labels.append(img.split(os.path.sep)[-2]) 

In [ ]:
print(labels)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
data = np.array(data, dtype="float")/255.

data=data.reshape(count,240,240,1)
le = LabelEncoder()
labels = le.fit_transform(labels)
labels = to_categorical(labels, 8)
print(labels)

In [ ]:
print(data.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.10, random_state=40)

In [ ]:
aug = ImageDataGenerator(
    rotation_range=(15),
    horizontal_flip=False,
    zoom_range=(0.9,1.2)
    )

In [ ]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D 
from keras.layers import Dense, Flatten ,GlobalMaxPool2D

In [ ]:
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(240, 240, 1),padding="same")) 
model.add(MaxPooling2D((5, 5)))

model.add(Conv2D(128, (3, 3),activation='relu'))
model.add(MaxPooling2D((3, 3)))

model.add(Conv2D(256, (3 ,3), activation='relu') )
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(512, (3 ,3), activation='relu') )
model.add(MaxPooling2D((2, 2)))


model.add(Flatten())
model.add(Dropout(0.5))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])

In [ ]:
model.fit(x=aug.flow(X_train, y_train,batch_size=32),batch_size=32, epochs=50, validation_data=aug.flow(X_test, y_test))

In [ ]:
model.save('F:/WorkSpace/Sign_Language_Testing/Words_testing/st_end/models/edge/sobel_9_4_conv')